<a href="https://colab.research.google.com/github/deokwoo-han/abc/blob/master/10_06_%EC%97%B0%EA%B4%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
 
# 데이터 파일(객체)이 어느정도 사이즈(MB) 인지 확인 하는 함수.
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

%matplotlib inline
%config InlineBackend.figure_config = 'retina'

#pd.options.mode.chained_assignment = None  # default='warn'

###1. 구글 드라이버 파일 가져오기(시행자 경로에 해당 파일이 있어야 함)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pwd cd

/content


###2. 파일 불러와서 데이터 프레임으로 만들기

In [4]:
file_path = './drive/MyDrive/datas'
order_products_train = pd.read_csv(file_path + "/order_products__train.csv")
order_products_prior = pd.read_csv(file_path + "/order_products__prior.csv")
order = pd.read_csv(file_path + "/orders.csv")
products = pd.read_csv(file_path + "/products.csv")
aisles = pd.read_csv(file_path + "/aisles.csv")
departments = pd.read_csv(file_path + "/departments.csv")

###3. order_products_prior 데이터프레임 용량 확인

In [5]:
def mem_usage(order_products_prior):
    if isinstance(order_products_prior,pd.DataFrame):
        usage_b = order_products_prior.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = order_products_prior.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [6]:
mem_usage(order_products_prior)

'989.82 MB'

###4. order_products_prior int타입 데이터 용량 줄인 후 전체 확인

In [7]:
def int_memory_reduce(order_products_prior) :
    data_int = order_products_prior.select_dtypes(include=['int'])
    converted_int = data_int.apply(pd.to_numeric,downcast='unsigned')
    print(f"Before : {mem_usage(data_int)} -> After : {mem_usage(converted_int)}")
    order_products_prior[converted_int.columns] = converted_int
    return order_products_prior

In [8]:
int_memory_reduce(order_products_prior)

Before : 989.82 MB -> After : 247.46 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [9]:
print('order_products_prior -- dimensions: {0};   size: {1}'.format(order_products_prior.shape, size(order_products_prior)))
display(order_products_prior.head())

order_products_prior -- dimensions: (32434489, 4);   size: 259.48 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


###5. orders 데이터 프레임으로 바꿔서 order_id와 product_id -> item_id만 남겨놓기

In [10]:
orders = order_products_prior.set_index('order_id')['product_id'].rename('item_id')
print('dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

dimensions: (32434489,);   size: 324.34 MB;   unique_orders: 3214874;   unique_items: 49677


In [11]:
orders

order_id
2          33120
2          28985
2           9327
2          45918
2          30035
           ...  
3421083    39678
3421083    11352
3421083     4600
3421083    24852
3421083     5020
Name: item_id, Length: 32434489, dtype: uint16

In [12]:
orders.shape

(32434489,)

In [13]:
orders.dtypes

dtype('uint16')

###6. 연관 규칙 함수들

In [14]:
# 단일 제품 또는 제품 집합 빈도수 반환
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")
     
# 고유 주문번호 갯수 반환
def order_count(order_item):
    return len(set(order_item.index))
 
# 한번에 한 제품 집합을 생성하는 generator 반환
def get_item_pairs(order_item):
    order_item = order_item.reset_index().values
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
               
        for item_pair in combinations(item_list, 2):
            yield item_pair            
 
# 제품에 대한 빈도수 및 지지도 반환
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))
 
# 제품 이름 반환
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))   
    return rules[columns]

In [15]:
# 주문 정보(주문번호를 인덱스로 하고 상품번호를 Value로하는 Series)와 최소 지지도를 입력받아 연관 규칙 반환
def association_rules(order_item, min_support):
 
    print("Starting order_item: {:22d}".format(len(order_item)))
 
    # 빈도수와 지지도를 계산
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100
 
    # 최소 지지도를 만족하지 못하는 제품은 제외
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]
 
    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))
 
    # 2개 미만의 제품 주문 정보는 제외
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]
 
    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))
 
    # 빈도수와 지지도를 다시 계산
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) * 100
 
    # 제품 집합에 대한 generator를 생성
    item_pair_gen          = get_item_pairs(order_item)
 
    # 제품 집합의 빈도수와 지지도를 계산
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100
 
    print("Item pairs: {:31d}".format(len(item_pairs)))
 
    # 최소 지지도를 만족하지 못하는 제품 집합을 제외
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]
 
    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))
 
    # 계산된 연관 규칙을 계산된 지표들과 함께 테이블로 생성
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
     
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
     
    # 향상도를 내림차순으로 정렬하여 연관 규칙 결과를 반환
    return item_pairs.sort_values('lift', ascending=False)

###7. 찾아낸 연관규칙 개수와 찾는 데 걸린 시간

In [16]:
%%time
rules = association_rules(orders, 0.01)

Starting order_item:               32434489
Items with support >= 0.01:           10906
Remaining order_item:              29843570
Remaining orders with 2+ items:     3013325
Remaining order_item:              29662716
Item pairs:                        30622410
Item pairs with support >= 0.01:      48751

CPU times: user 5min 18s, sys: 5.63 s, total: 5min 23s
Wall time: 5min 23s


###8. A제품과 B제품 간의 연관 규칙 결과 테이블(향상도 순)

freqAB: 제품 A, B 동시 구매 빈도수

supportAB: 전체 주문 내역 중 제품 A, B 동시 구매비율

freqA: 제품 A 구매 빈도수

supportA: 전체 주문 내역 중 제품 A 구매비율

freqB: 제품 B 구매 빈도수

supportB: 전체 주문 내역 중 제품 B 구매비율

confidenceAtoB: 제품 A가 구매되었을 때 제품 B가 추가로 구매될 조건부확률

confidenceBtoA: 제품 B가 구매되었을 때 제품 A가 추가로 구매될 조건부확률

lift: 동시 구매 여부를 알기 위해 서로 간의 연관성을 파악하는 향상도(1보다 높을 경우 itemA, itemB 두 품목 간의 긍정적 관계, 낮을 경우 두 품목 간의 부정적 관계, 1일 경우 관계 없음)



In [17]:
item_name   = pd.read_csv(file_path + "/products.csv")
item_name   = item_name.rename(columns={'product_id':'item_id', 'product_name':'item_name'})
rules_final = merge_item_name(rules, item_name).sort_values('lift', ascending=False)
display(rules_final)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,0.010155,1163,0.038595,839,0.027843,0.263113,0.364720,9.449868
1,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,318,0.010553,1809,0.060033,879,0.029170,0.175788,0.361775,6.026229
3,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,349,0.011582,1518,0.050376,1249,0.041449,0.229908,0.279424,5.546732
9,Nonfat Strawberry With Fruit On The Bottom Gre...,"0% Greek, Blueberry on the Bottom Yogurt",409,0.013573,1666,0.055288,1391,0.046162,0.245498,0.294033,5.318230
10,Organic Grapefruit Ginger Sparkling Yerba Mate,Cranberry Pomegranate Sparkling Yerba Mate,351,0.011648,1731,0.057445,1149,0.038131,0.202773,0.305483,5.317849
...,...,...,...,...,...,...,...,...,...,...,...
7271,Organic Strawberries,Strawberries,640,0.021239,263416,8.741706,141805,4.705931,0.002430,0.004513,0.000516
6763,Organic Hass Avocado,Organic Avocado,464,0.015398,212785,7.061469,176241,5.848722,0.002181,0.002633,0.000373
4387,Organic Avocado,Organic Hass Avocado,443,0.014701,176241,5.848722,212785,7.061469,0.002514,0.002082,0.000356
2596,Banana,Bag of Organic Bananas,654,0.021704,470096,15.600574,376367,12.490090,0.001391,0.001738,0.000111


In [18]:
rules_final.head(20)

,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,306,0.010155,1163,0.038595,839,0.027843,0.263113,0.364720,9.449868
1,Grain Free Chicken Formula Cat Food,Grain Free Turkey Formula Cat Food,318,0.010553,1809,0.060033,879,0.029170,0.175788,0.361775,6.026229
3,Organic Fruit Yogurt Smoothie Mixed Berry,Apple Blueberry Fruit Yogurt Smoothie,349,0.011582,1518,0.050376,1249,0.041449,0.229908,0.279424,5.546732
9,Nonfat Strawberry With Fruit On The Bottom Gre...,"0% Greek, Blueberry on the Bottom Yogurt",409,0.013573,1666,0.055288,1391,0.046162,0.245498,0.294033,5.318230
10,Organic Grapefruit Ginger Sparkling Yerba Mate,Cranberry Pomegranate Sparkling Yerba Mate,351,0.011648,1731,0.057445,1149,0.038131,0.202773,0.305483,5.317849
11,Baby Food Pouch - Roasted Carrot Spinach & Beans,"Baby Food Pouch - Butternut Squash, Carrot & C...",332,0.011018,1503,0.049878,1290,0.042810,0.220892,0.257364,5.159830
12,Unsweetened Whole Milk Mixed Berry Greek Yogurt,Unsweetened Whole Milk Blueberry Greek Yogurt,438,0.014535,1622,0.053828,1621,0.053794,0.270037,0.270204,5.019798
23,Uncured Cracked Pepper Beef,Chipotle Beef & Pork Realstick,410,0.013606,1839,0.061029,1370,0.045465,0.222947,0.299270,4.903741
24,Organic Mango Yogurt,Organic Whole Milk Washington Black Cherry Yogurt,334,0.011084,1675,0.055586,1390,0.046128,0.199403,0.240288,4.322777
2,Grain Free Chicken Formula Cat Food,Grain Free Turkey & Salmon Formula Cat Food,391,0.012976,1809,0.060033,1553,0.051538,0.216142,0.251771,4.193848


###9. 데이터프레임 CSV화

In [19]:
rules_final.to_csv('sample1.csv')

향상도를 통한 연관 규칙 분석을 이용해 브랜드 있는 유기농(오가닉) 제품을 살 경우 일반 제품을 사는 경우와 그 역의 사례 또한 극히 적음(아보카도 예외)을 불 수 있으며, 반대로 같은 이름으로 시작하는 유기농 제품이나 일반 제품을 동시 구매하는 사례는 높은 향상도 순위를 보였다. 

결론적으로 유기농 브랜드 제품을 사는 소비자군과 일반 제품을 사는 소비자군이 크게 겹쳐지지 않는 것으로 보인다.